# Thermal Convection

This example solves 2D dimensionless isoviscous thermal convection with a Rayleigh number of $10^7$, see Blankenbach *et al.* 1989 for details.

**References**

B. Blankenbach, F. Busse, U. Christensen, L. Cserepes, D. Gunkel, U. Hansen, H. Harder, G. Jarvis, M. Koch, G. Marquart, D. Moore, P. Olson, H. Schmeling and T. Schnaubelt. A benchmark comparison for mantle convection codes. Geophysical Journal International, 98, 1, 23–38, 1989
http://onlinelibrary.wiley.com/doi/10.1111/j.1365-246X.1989.tb05511.x/abstract

In [1]:
#|  echo: false  # Hide in html version

# This is required to fix pyvista 
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio
nest_asyncio.apply()

In [2]:
#| output: false # Suppress warnings in html version

import underworld3 as uw
import numpy as np
import sympy

[MacBook-Pro.local:32713] shmem: mmap: an error occurred while determining whether or not /var/folders/0j/bnxlsh897sl6b1rv06fnt5r80000gp/T//ompi.MacBook-Pro.502/jf.0/2217738240/sm_segment.MacBook-Pro.502.84300000.0 could be created.


#### Setup scaling of the model

In [3]:
u = uw.scaling.units

### make scaling easier
ndim = uw.scaling.non_dimensionalise
nd   = uw.scaling.non_dimensionalise
dim  = uw.scaling.dimensionalise 



boxHeight = 1000. * u.kilometer
boxLength = 2000. * u.kilometer

tempMin = 273.15 * u.degK
tempMax = 1273.15 * u.degK

refViscosity = 1e23 * u.pascal * u.second

KL = boxHeight
KT = (tempMax - tempMin)
Kt = 1.0 * u.megayear
KM = refViscosity * KL * Kt



scaling_coefficients  = uw.scaling.get_coefficients()
scaling_coefficients["[length]"] = KL
scaling_coefficients["[time]"]= Kt
scaling_coefficients["[mass]"]= KM
scaling_coefficients["[temperature]"]= KT

scaling_coefficients

[mass],3.1557599999999994e+42 kilogram
[length],1000000.0 meter
[temperature],1000.0000000000001 kelvin
[time],31557600000000.0 second
[substance],1.0 mole


#### Setup the mesh

In [4]:
# mesh = uw.meshing.UnstructuredSimplexBox(
#             minCoords=(nd(0. * u.centimeter), nd(0. * u.centimeter)), 
#             maxCoords=(nd(20. * u.centimeter), nd(10. * u.centimeter)),
#             cellSize=1 / 12
# )

mesh = uw.meshing.StructuredQuadBox(
                minCoords=(0., 0.), 
                  maxCoords=(nd(2000. * u.kilometer), nd(1000. * u .kilometer)),
            elementRes=(32,32)
)


mesh.view()

Structured box element resolution 32 32


Mesh # 0: .meshes/uw_structuredQuadBox_minC(0.0, 0.0)_maxC(2.0, 1.0).msh

No variables are defined on the mesh

| Boundary Name            | ID    | Min Size | Max Size |
| ------------------------------------------------------ |
| Bottom                   | 11    | 63       | 63       |
| Top                      | 12    | 63       | 63       |
| Right                    | 13    | 63       | 63       |
| Left                     | 14    | 63       | 63       |
| Null_Boundary            | 666   | 1089     | 1089     |
| All_Boundaries           | 1001  | 128      | 128      |
| All_Boundaries           | 1001  | 128      | 128      |
| UW_Boundaries            | --    | 1469     | 1469     |
| ------------------------------------------------------ |


DM Object: uw_.meshes/uw_structuredQuadBox_minC(0.0, 0.0)_maxC(2.0, 1.0).msh 1 MPI process
  type: plex
uw_.meshes/uw_structuredQuadBox_minC(0.0, 0.0)_maxC(2.0, 1.0).msh in 2 dimensions:
  Number

In [5]:
from mpi4py import MPI

if MPI.COMM_WORLD.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)

    pl = pv.Plotter(window_size=(1000, 500), shape=(1, 1))


    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        use_transparency=False,
        opacity=1,
        show_scalar_bar=True,
    )

    pl.show()


Widget(value='<iframe src="http://localhost:62395/index.html?ui=P_0x324f625c0_0&reconnect=auto" class="pyvista…

In [6]:
# mesh variables
### for stokes
p_soln = uw.discretisation.MeshVariable(varname="p", mesh=mesh, num_components=1, degree=0, continuous=False )
v_soln = uw.discretisation.MeshVariable(varname="v", mesh=mesh, num_components=2, degree=1)
#### for advDiff
T_soln = uw.discretisation.MeshVariable(varname="T", mesh=mesh, num_components=1, degree=1)


### The Stokes solver

There are a number of pre-defined *solver systems* defined in `underworld3` 
which are templates for orchestrating the underlying PETSc objects. 
A solver requires us to define the unknown in the form of `meshVariables`, 
provide boundary conditions, a constitutive model, 
and provide `uw.functions` to define constitutive
properties, and driving terms.

We will use the `Stokes` solver to solve for the body force (forcing term) and viscous drag (flux term).

The solver classes themselves are documented, so we can figure out what 
is needed before we define anything:


In [7]:
uw.systems.Stokes.view()


This class provides functionality for a discrete representation
of the Stokes flow equations assuming an incompressibility
(or near-incompressibility) constraint.

$$
\nabla \cdot
        \color{Blue}{\underbrace{\Bigl[
                \boldsymbol{\tau} -  p \mathbf{I} \Bigr]}_{\mathbf{F}}} =
        \color{Maroon}{\underbrace{\Bigl[ \mathbf{f} \Bigl] }_{\mathbf{h}}}
$$

$$
\underbrace{\Bigl[ \nabla \cdot \mathbf{u} \Bigr]}_{\mathbf{h}_p} = 0
$$

The flux term is a deviatoric stress ( $\boldsymbol{\tau}$ ) related to velocity gradients
  ( $\nabla \mathbf{u}$ ) through a viscosity tensor, $\eta$, and a volumetric (pressure) part $p$

$$
    \mathbf{F}: \quad \boldsymbol{\tau} = \frac{\eta}{2}\left( \nabla \mathbf{u} + \nabla \mathbf{u}^T \right)
$$

The constraint equation, $\mathbf{h}_p = 0$ gives incompressible flow by default but can be set
to any function of the unknown  $\mathbf{u}$ and  $\nabla\cdot\mathbf{u}$

## Properties

  - The unknowns are velocities $\mathbf{u}$ and a pressure-like constraint parameter $\mathbf{p}$

  - The viscosity tensor, $\boldsymbol{\eta}$ is provided by setting the `constitutive_model` property to
one of the scalar `uw.constitutive_models` classes and populating the parameters.
It is usually a constant or a function of position / time and may also be non-linear
or anisotropic.

  - $\mathbf f$ is a volumetric source term (i.e. body forces)
  and is set by providing the `bodyforce` property.

  - An Augmented Lagrangian approach to application of the incompressibility
constraint is to penalise incompressibility in the Stokes equation by adding
$ \lambda \nabla \cdot \mathbf{u} $ when the weak form of the equations is constructed.
(this is in addition to the constraint equation, unlike in the classical penalty method).
This is activated by setting the `penalty` property to a non-zero floating point value which adds
the term in the `sympy` expression.

  - A preconditioner is usually required for the saddle point system and this is provided
though the `saddle_preconditioner` property. The default choice is $1/\eta$ for a scalar viscosity function.

## Notes

  - For problems with viscoelastic behaviour, the flux term contains the stress history as well as the
    stress and this term is a Lagrangian quantity that has to be tracked on a particle swarm.

  - The interpolation order of the `pressureField` variable is used to determine the integration order of
the mixed finite element method and is usually lower than the order of the `velocityField` variable.

  - It is possible to set discontinuous pressure variables by setting the `p_continous` option to `False`



### Problem setup

In [8]:
# Setup stokes
stokes = uw.systems.Stokes(mesh, velocityField=v_soln, pressureField=p_soln)

### Constitutive Models

Most of the solvers require a constitutive model to be provided and its
parameters populated. This is to allow flexibility in defining / redefining 
solvers during a model calculation.

We need a viscous flow model. We can look at the documentation first. 

In [9]:
uw.constitutive_models.ViscousFlowModel.view()


### Viscous Flow Model

$$\tau_{ij} = \eta_{ijkl} \cdot \frac{1}{2} \left[ \frac{\partial u_k}{\partial x_l} + \frac{\partial u_l}{\partial x_k} \right]$$

where $\eta$ is the viscosity, a scalar constant, `sympy` function, `underworld` mesh variable or
any valid combination of those types. This results in an isotropic (but not necessarily homogeneous or linear)
relationship between $\tau$ and the velocity gradients. You can also supply $\eta_{IJ}$, the Mandel form of the
constitutive tensor, or $\eta_{ijkl}$, the rank 4 tensor.

The Mandel constitutive matrix is available in `viscous_model.C` and the rank 4 tensor form is
in `viscous_model.c`.




In [10]:
stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel

##### We use the material index to map a viscosity

In [11]:
eta = uw.function.expression(
                    r'\eta',
                    sym=nd(1e23*u.pascal*u.second),
                    description="model viscosity"
                        )


In [12]:
stokes.constitutive_model.Parameters.shear_viscosity_0 = eta
stokes.saddle_preconditioner = 1 / eta
stokes.constitutive_model.view()

**Class**: <class 'underworld3.constitutive_models.ViscousFlowModel'>

This consititutive model is formulated for 2 dimensional equations

<IPython.core.display.Latex object>

#### Setup temperature-dependent density

In [13]:
alpha = uw.function.expression(
                    r'\alpha',
                    sym=nd(2.5e-5/u.degK),
                    description="model thermal expansivity"
                        )

rho_0 = uw.function.expression(
                    r'\rho',
                    sym=nd(4000*u.kilogram/u.meter**3),
                    description="model density"
                        )

T_0 = uw.function.expression(
                    r'T_{0}',
                    sym=nd(273.15*u.kelvin),
                    description="reference temperature at surface"
                        )

t_term = alpha.sym * (T_soln.sym[0] - T_0)



density_fn = rho_0.sym * (1 - t_term)



gravity = uw.function.expression(
                    r'g',
                    sym=nd(9.81*u.meter/u.second**2),
                    description="gravity"
                        )

In [14]:
density_fn

3.1688087814029e-23*{ T_{0} \hspace{ 0.07pt } } - 3.1688087814029e-23*{T}(N.x, N.y) + 1.26752351256116e-21

In [15]:
stokes.bodyforce = sympy.Matrix([0, -1*density_fn*gravity])

In [16]:
stokes.view()

**Class**: <class 'underworld3.systems.solvers.SNES_Stokes'>

### Saddle point system solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

#### Boundary Conditions

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

### Setting up boundary coniditions
Free slip everywhere

In [17]:
v_0 = uw.function.expression(
                    r'v_{0}',
                    sym=0.,
                    description="zero velocity component"
)

In [18]:
# Add boundary conditions
stokes.add_essential_bc([None, v_0], "Bottom")
stokes.add_essential_bc([None, v_0], "Top"  )

stokes.add_essential_bc([v_0, None], "Left")
stokes.add_essential_bc([v_0, None], "Right"  )




#### Setup the advection-diffusion solver

In [19]:
advDiff = uw.systems.AdvDiffusion(mesh, T_soln, v_soln)
advDiff.constitutive_model = uw.constitutive_models.DiffusionModel

In [20]:
# Set the diffusivity
kappa = uw.function.expression(
                    r'\upkappa',
                    sym=nd(1.0e-6 * u.meter**2 / u.second),
                    description="thermal diffusivity"
                        )

advDiff.constitutive_model.Parameters.diffusivity = kappa

In [21]:
T_t = uw.function.expression(
                    r'T_t',
                    sym=nd(tempMin),
                    description="top temperature"
                        )

T_b = uw.function.expression(
                    r'T_b',
                    sym=nd(tempMax),
                    description="base temperature"
                        )

In [22]:
# Add boundary conditions
advDiff.add_essential_bc([T_b], "Bottom")
advDiff.add_essential_bc([T_t], "Top"  )

In [23]:
nd(tempMin)

0.27314999999999995

In [24]:
pertStrength = nd(200. * u.kilometer)
deltaTemp = nd(tempMax) - nd(tempMin)

x, y = mesh.X

pertCoeff = sympy.cos( sympy.pi * x ) * sympy.sin( sympy.pi * y )

tempFn = nd(tempMin) + deltaTemp*(nd(boxHeight) - y) + pertStrength * pertCoeff



with mesh.access(T_soln):
    T_soln.data[:,0] = 0.
    T_soln.data[:,0] = uw.function.evaluate(tempFn, T_soln.coords)
    print(T_soln.data.max())
                    
    T_soln.data[T_soln.data[:,0] > nd(tempMax)] = nd(tempMax)
    T_soln.data[T_soln.data[:,0] < nd(tempMin)] = nd(tempMin)


1.27315


In [25]:
from mpi4py import MPI

if MPI.COMM_WORLD.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, T_soln.sym)

    pl = pv.Plotter(window_size=(1000, 500), shape=(1, 1))


    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1,
        show_scalar_bar=True,
    )

    pl.show()


Widget(value='<iframe src="http://localhost:62395/index.html?ui=P_0x3272ea590_1&reconnect=auto" class="pyvista…

#### Check the system setup by using view()

In [26]:
advDiff.view()

**Class**: <class 'underworld3.systems.solvers.SNES_AdvectionDiffusion'>

### Poisson system solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Bottom | $\left[\begin{matrix}{ T_b \hspace{ 0.2pt } }\end{matrix}\right]  $ | 
| **essential** | Top | $\left[\begin{matrix}{ T_t \hspace{ 0.19pt } }\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [27]:
stokes.view()

**Class**: <class 'underworld3.systems.solvers.SNES_Stokes'>

### Saddle point system solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

#### Boundary Conditions

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Bottom | $\left[\begin{matrix}\infty & { v_{0} \hspace{ 0.14pt } }\end{matrix}\right]  $ | 
| **essential** | Top | $\left[\begin{matrix}\infty & { v_{0} \hspace{ 0.14pt } }\end{matrix}\right]  $ | 
| **essential** | Left | $\left[\begin{matrix}{ v_{0} \hspace{ 0.14pt } } & \infty\end{matrix}\right]  $ | 
| **essential** | Right | $\left[\begin{matrix}{ v_{0} \hspace{ 0.14pt } } & \infty\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

##### Solve the system

In [28]:
stokes.petsc_options["snes_converged_reason"] = None
advDiff.petsc_options["snes_converged_reason"] = None

In [29]:
time = 0.
step = 0

In [30]:
# Solve the system n times
for nsteps in range(5):

    print(f'step = {step}, time = {dim(time, u.megayear)}')
    
    stokes.solve(zero_init_guess=True)
    dt = stokes.estimate_dt()
    ### advect particles
    advDiff.solve(timestep=dt)
    
    step += 1
    time += dt

step = 0, time = 0.0 megayear
  Nonlinear Solver_4_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  Nonlinear Solver_21_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step = 1, time = 22.11919963402177 megayear
  Nonlinear Solver_4_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  Nonlinear Solver_21_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step = 2, time = 41.44260821594348 megayear
  Nonlinear Solver_4_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  Nonlinear Solver_21_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step = 3, time = 58.57190491417184 megayear
  Nonlinear Solver_4_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  Nonlinear Solver_21_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
step = 4, time = 74.02477537435827 megayear
  Nonlinear Solver_4_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  Nonlinear Solver_21_ solve converged due 

### Visualise the temperature field

In [40]:
from mpi4py import MPI

if MPI.COMM_WORLD.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["V"] = uw.visualisation.vector_fn_to_pv_points(pvmesh, v_soln.sym)
    pvmesh.point_data["T"] = uw.visualisation.scalar_fn_to_pv_points(pvmesh, T_soln.sym)

    velocity_points = uw.visualisation.meshVariable_to_pv_cloud(v_soln)
    velocity_points.point_data["V"] = uw.visualisation.vector_fn_to_pv_points(velocity_points, v_soln.sym)


    pl = pv.Plotter(window_size=(1000, 500), shape=(1, 1))


    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1,
        show_scalar_bar=True,
    )

    arrows = pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=2e2, opacity=0.1, show_scalar_bar=False)

    pl.show()

Widget(value='<iframe src="http://localhost:62395/index.html?ui=P_0x333407d90_11&reconnect=auto" class="pyvist…